In [1]:
import pandas as pd
import glob
import json

In [4]:
json_folder = glob.glob("data/raw_json/*/*.json")

In [6]:
sesiones = []
votaciones = []
grupos_parlamentarios = {}
diputados = {}
votos = []
grupo_id_counter = 0
diputado_id_counter = 0

In [7]:
for json_file in json_folder:
    with open(json_file, "r", encoding="utf-8") as json_file:
        # Load the JSON file
        # Assuming the JSON file contains a list of dictionaries
        # If the structure is different, adjust accordingly
        # For example, if it's a single dictionary, you can just load it directly
        #
        data = json.load(json_file)
        # Extract session data
        sesion_id = data["informacion"]["sesion"]
        if sesion_id not in [s["id_sesion"] for s in sesiones]:
            sesiones.append({
                "id_sesion": sesion_id,
                "fecha": data["informacion"]["fecha"],
                "numero_sesion": data["informacion"]["sesion"]
            })

        # Extract votacion data
        votacion_id = len(votaciones) + 1
        votaciones.append({
            "id_votacion": votacion_id,
            "id_sesion": sesion_id,
            "numero_votacion": data["informacion"]["numeroVotacion"],
            "titulo": data["informacion"]["titulo"],
            "texto_expediente": data["informacion"]["textoExpediente"],
            "presentes": data["totales"]["presentes"],
            "a_favor": data["totales"]["afavor"],
            "en_contra": data["totales"]["enContra"],
            "abstenciones": data["totales"]["abstenciones"],
            "no_votan": data["totales"]["noVotan"]
        })

        # Extract grupos parlamentarios and diputados
        for votacion in data["votaciones"]:
            grupo = votacion.get("grupo", "Unknown")
            if grupo not in grupos_parlamentarios:
                grupos_parlamentarios[grupo] = grupo_id_counter
                grupo_id_counter += 1

            diputado = votacion.get("diputado", "Unknown")
            if diputado not in diputados:
                diputados[diputado] = {
                    "id_diputado": diputado_id_counter,
                    "nombre": diputado,
                    "id_grupo": grupos_parlamentarios[grupo]
                }
                diputado_id_counter += 1

            # Extract votos
            votos.append({
                "id_voto": len(votos) + 1,
                "id_votacion": votacion_id,
                "id_diputado": diputados[diputado]["id_diputado"],
                "asiento": votacion.get("asiento", ""),
                "voto": votacion["voto"]
            })

# Convert data to DataFrames
sesiones_df = pd.DataFrame(sesiones)
votaciones_df = pd.DataFrame(votaciones)
grupos_parlamentarios_df = pd.DataFrame([
{"id_grupo": id_grupo, "nombre": nombre}
for nombre, id_grupo in grupos_parlamentarios.items()
])
diputados_df = pd.DataFrame(diputados.values())
votos_df = pd.DataFrame(votos)

In [8]:
# format date of sesiones_df to YYYY-MM-DD
sesiones_df["fecha"] = pd.to_datetime(sesiones_df["fecha"], format="%d/%m/%Y").dt.strftime("%Y-%m-%d")

In [10]:
# Save DataFrames to CSV files in data/processed folder (create if not exists)
import os
os.makedirs("data/processed", exist_ok=True)
sesiones_df.to_csv("data/processed/sesiones.csv", index=False, encoding="utf-8")
votaciones_df.to_csv("data/processed/votaciones.csv", index=False, encoding="utf-8")
grupos_parlamentarios_df.to_csv("data/processed/grupos_parlamentarios.csv", index=False, encoding="utf-8")
diputados_df.to_csv("data/processed/diputados.csv", index=False, encoding="utf-8")
votos_df.to_csv("data/processed/votos.csv", index=False, encoding="utf-8")